In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.basketball-reference.com/boxscores/202310240DEN.html'


def get_page_data(url):
    data_source = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    return BeautifulSoup(data_source.content, 'html.parser')

page = get_page_data(url)

In [41]:
tables = page.find_all('table')
basic_team_A = tables[0]
advanced_team_A = tables[7]
basic_team_H = tables[8]
advanced_team_H = tables[15]

In [42]:
def get_game_details(page):
    details = page.find(id='content').find('h1').get_text()
    return details.replace(' ', '').replace(',','_')


game_details = get_game_details(page)
game_details

'LosAngelesLakersatDenverNuggetsBoxScore_October24_2023'

In [43]:
def get_basic_table_headers(table):
    return [th.getText().split('\n') for th in table.find_all('tr', limit=5)[1:2]][0][1:-1]

In [44]:
def get_Basic_table_rows(table):
    return [td for td in table.find('tbody').find_all('tr')]



In [45]:
def get_basic_table_data(rows):
    player_stats = [[td.getText() for td in rows[i].find_all('td')]for i in range(len(rows))]
    player_names = [[td.getText() for td in rows[i].find_all('a')]for i in range(len(rows))]
    return player_names ,player_stats



In [46]:
def make_basic_table_rows(player_names, player_stats):
    player_names[5].insert(5, 'player')
    for i in range(len(player_names)):
        player = player_names[i][0]
        (player_stats[i].insert(0,player))

def make_basic_table_pandas(basic_table):
    basic_table_player_names, basic_table_player_stats = get_basic_table_data(rows = get_Basic_table_rows(basic_table))
    headers = get_basic_table_headers(basic_table)
    make_basic_table_rows(basic_table_player_names, basic_table_player_stats)
    return pd.DataFrame(basic_table_player_stats, columns=headers)
    
basic_table_home = make_basic_table_pandas(basic_team_H)
basic_table_away = make_basic_table_pandas(basic_team_A)

In [47]:
basic_table_away

,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
0,D'Angelo Russell,36:11,4,12,.333,2,5,.400,1,2,...,0,4,4,7,1,0,3,3,11,+1
1,Anthony Davis,34:09,6,17,.353,1,2,.500,4,4,...,1,7,8,4,0,2,2,3,17,-17
2,Austin Reaves,31:20,4,11,.364,1,2,.500,5,7,...,4,4,8,4,2,0,2,2,14,-14
3,Taurean Prince,29:53,6,8,.750,4,6,.667,2,2,...,1,2,3,1,0,1,1,0,18,-14
4,LeBron James,29:00,10,16,.625,1,4,.250,0,1,...,1,7,8,5,1,0,0,1,21,+7
5,player,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Gabe Vincent,22:18,3,8,.375,0,4,.000,0,0,...,1,0,1,2,1,0,2,3,6,-17
7,Cam Reddish,17:38,2,4,.500,1,2,.500,2,2,...,2,2,4,0,0,1,0,2,7,+7
8,Christian Wood,15:28,3,4,.750,0,1,.000,1,2,...,1,3,4,0,0,0,1,1,7,+2
9,Rui Hachimura,14:39,3,10,.300,0,3,.000,0,0,...,2,1,3,0,0,0,0,2,6,-8


In [48]:
basic_table_home

,Starters,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
0,Nikola Jokić,36:16,12,22,.545,3,5,.600,2,4,...,3,10,13,11,1,1,2,2,29,+15
1,Kentavious Caldwell-Pope,36:15,8,12,.667,2,3,.667,2,2,...,1,1,2,1,3,1,3,5,20,+10
2,Aaron Gordon,34:59,7,11,.636,1,2,.500,0,0,...,2,5,7,5,2,1,0,0,15,+6
3,Jamal Murray,34:15,8,13,.615,3,5,.600,2,2,...,0,2,2,6,0,1,1,3,21,+3
4,Michael Porter Jr.,30:08,5,13,.385,2,9,.222,0,0,...,2,10,12,2,2,0,0,1,12,+12
5,player,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Reggie Jackson,24:04,3,8,.375,2,5,.400,0,0,...,0,3,3,1,1,0,2,0,8,+11
7,Christian Braun,19:20,2,5,.400,0,1,.000,1,2,...,1,2,3,2,0,1,1,1,5,+5
8,Zeke Nnaji,11:44,1,3,.333,0,1,.000,2,2,...,0,0,0,1,0,0,1,2,4,-3
9,Peyton Watson,10:50,1,3,.333,1,3,.333,0,0,...,0,0,0,0,0,1,1,1,3,+1


In [49]:
def write_basic_tables_to_file(details,basic_table_home,basic_table_away):
    path = './box_score_tables_csv'
    home_path = f'{path}/basic_table_H_{details}.csv'
    away_path = f'{path}/basic_table_A_{details}.csv'
    pd.DataFrame.to_csv(basic_table_home,home_path)
    pd.DataFrame.to_csv(basic_table_away,away_path)

In [50]:
write_basic_tables_to_file(details=game_details,basic_table_home=basic_table_home,basic_table_away=basic_table_away)

./box_score_tables_csv/basic_table_H_LosAngelesLakersatDenverNuggetsBoxScore_October24_2023.csv
./box_score_tables_csv/basic_table_A_LosAngelesLakersatDenverNuggetsBoxScore_October24_2023.csv
